### Train nlu model
!python -m rasa_nlu.train -c nlu_config.yml --data data/nlu.md -o models --fixed_model_name nlu --project current --verbose

### Train dialogue model
!python -m rasa_core.train -d domain.yml -s data/stories.md -o models/dialogue

In [ ]:
python -m rasa_core.train -d domain.yml -s stories.md -o models/dialogue --epochs 200

In [15]:
import IPython
from IPython.display import clear_output, HTML, display
from rasa_core.agent import Agent
from rasa_core.interpreter import RasaNLUInterpreter
import time

interpreter = RasaNLUInterpreter('models/nlu/default/chat')
messages = ["Hi! you can chat in this window. Type 'stop' to end the conversation."]
agent = Agent.load('models/dialogue', interpreter=interpreter)

def chatlogs_html(messages):
    messages_html = ""
    for m in messages:
        if m.endswith('.jpg'):
             messages_html += "<img src={}, alt='Tiger pub'></img>".format(m)
        else:
             messages_html += "<p>{}</p>".format(m)
    chatbot_html = """<div class="chat-window"> {}</div>""".format(messages_html)
    return chatbot_html


while True:
    clear_output()
    display(HTML(chatlogs_html(messages)))
    time.sleep(0.3)
    a = input()
    messages.append(a)
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for r in responses:
        display(r.keys())
        key = 'image' if 'image' in r.keys() else 'text'
        messages.append(r.get(key))                    

stop


## Indetify intent

In [2]:
from rasa_core.agent import Agent
from rasa_core.interpreter import RasaNLUInterpreter
# Loading the Agent
rasaNLU = RasaNLUInterpreter("models/nlu/default/chat")
agent = Agent.load("models/dialogue", interpreter= rasaNLU)

INFO:tensorflow:Restoring parameters from models/nlu/default/chat\intent_classifier_tensorflow_embedding.ckpt


In [9]:
intent=rasaNLU.parse('what is the ETA of order id 12345?')
intent['intent']

{'name': 'order_status_eta', 'confidence': 0.8397842645645142}

In [10]:
intent['entities'][0]['entity']

'order_number'

In [ ]:
agent.handle_text('i want to order pizza')

## Visualize the stories

In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

from rasa_core.agent import Agent
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.policies.memoization import MemoizationPolicy

if __name__ == '__main__':
    agent = Agent("domain.yml",
                  policies=[MemoizationPolicy(), KerasPolicy()])

    agent.visualize("data/stories.md",
                    output_file="graph.png", max_history=2)

Processed Story Blocks: 100%|███| 16/16 [00:00<00:00, 420.97it/s, # trackers=1]


## Chatbot 2

In [1]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config
from rasa_nlu.model import Metadata, Interpreter
import os


def train_nlu (data, config_file, model_dir):
    # Load training nlu data
    training_data = load_data(data)
    # Load config file and create trainer object
    trainer = Trainer(config.load(config_file))
    # train the nlu model
    trainer.train(training_data)
    model_directory = trainer.persist(model_dir, fixed_model_name = 'chat')


def get_intent_confidence(text):
    # loading the interpreter 
    interpreter = Interpreter.load('/models/nlu/default/chat')
    # Predict intent
    intent=interpreter.parse(text)
    # Print the predicted intent dictionary with confidence
    return intent['intent']


def train_dialogue(path="python -m rasa_core.train -d domain.yml -s data/stories.md -o models/dialogue"):
    x=os.system(path)
    if x==0:
    	print("Model Trained Successfully")
    else:
    	print("Error, Please chcek the issue")

In [2]:
# Calling NLU training tunction
train_nlu('data/nlu.md', 'nlu_config.yml', 'models/nlu')

# Calling dialogue training function
train_dialogue()

Epochs: 100%|█████████| 300/300 [00:04<00:00, 68.82it/s, loss=0.159, acc=1.000]


Model Trained Successfully


In [22]:
from rasa_nlu.evaluate import run_evaluation
test_file, model_directory="data/nlu.md", "models/nlu/default/chat/"
def evaluate_nlu_model(test_file,model_directory):
    run_evaluation(test_file, model_directory)

In [23]:
evaluate_nlu_model(test_file, model_directory)

INFO:tensorflow:Restoring parameters from models/nlu/default/chat/intent_classifier_tensorflow_embedding.ckpt


In [11]:
from rasa_core.agent import Agent
from rasa_core.interpreter import RasaNLUInterpreter
# Loading the Agent
rasaNLU = RasaNLUInterpreter("models/nlu/default/chat")
agent = Agent.load("models/dialogue", interpreter= rasaNLU)

INFO:tensorflow:Restoring parameters from models/nlu/default/chat\intent_classifier_tensorflow_embedding.ckpt


In [24]:
intent=rasaNLU.parse('ETA of order no 12345?')
intent['intent']

{'name': 'order_status_eta', 'confidence': 0.9054824113845825}

In [25]:
intent['entities'][0]['entity'],intent['entities'][0]['value']

('order_number', '12345')

In [ ]:
import IPython
from IPython.display import clear_output, HTML, display
from rasa_core.agent import Agent
import time

from rasa_core.agent import Agent
from rasa_core.interpreter import RasaNLUInterpreter
from rasa_core.utils import EndpointConfig

core_endpoint_config = EndpointConfig(url="http://localhost:5055/webhook")

interpreter = RasaNLUInterpreter('models/nlu/default/chat')
messages = ["Hi! you can chat in this window. Type 'stop' to end the conversation."]
agent = Agent.load('models/dialogue', interpreter=interpreter, action_endpoint = core_endpoint_config)

def chatlogs_html(messages):
    messages_html = ""
    for m in messages:
        if m.endswith('.jpg'):
             messages_html += "<img src={}, alt='Tiger pub'></img>".format(m)
        else:
             messages_html += "<p>{}</p>".format(m)
    chatbot_html = """<div class="chat-window"> {}</div>""".format(messages_html)
    return chatbot_html


while True:
    clear_output()
    display(HTML(chatlogs_html(messages)))
    time.sleep(0.3)
    a = input()
    messages.append(a)
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for r in responses:
        display(r.keys())
        key = 'image' if 'image' in r.keys() else 'text'
        messages.append(r.get(key))                    

In [ ]:
What is the status of my order number

### API Call

In [2]:
# importing the requests library 
import requests 
  
# api-endpoint 
URL = "http://maps.googleapis.com/maps/api/geocode/json"
  
# location given here 
location = "delhi technological university"
  
# defining a params dict for the parameters to be sent to the API 
PARAMS = {'address':location} 
  
# sending get request and saving the response as response object 
r = requests.get(url = URL, params = PARAMS) 
  
# extracting data in json format 
data = r.json() 

print(data)
  
  
# extracting latitude, longitude and formatted address  
# of the first matching location 
latitude = data['results'][0]['geometry']['location']['lat'] 
longitude = data['results'][0]['geometry']['location']['lng'] 
formatted_address = data['results'][0]['formatted_address'] 
  
# printing the output 
print("Latitude:%s\nLongitude:%s\nFormatted Address:%s"
      %(latitude, longitude,formatted_address)) 

{'error_message': 'You must use an API key to authenticate each request to Google Maps Platform APIs. For additional information, please refer to http://g.co/dev/maps-no-account', 'results': [], 'status': 'REQUEST_DENIED'}


IndexError: list index out of range

In [19]:
import requests
req = requests.get('https://github.com/timeline.json')

In [30]:
import json
req_dict = json.loads(req.text)

In [31]:
req_dict['message']

'Hello there, wayfaring stranger. If you’re reading this then you probably didn’t see our blog post a couple of years back announcing that this API would go away: http://git.io/17AROg Fear not, you should be able to get what you need from the shiny new Events API instead.'

In [8]:
import requests
req = requests.get('https://www.edureka.co/')

In [10]:
req.encoding # returns 'utf-8'

'UTF-8'

In [11]:
req.status_code # returns 200

200

In [12]:
req.cookies

<RequestsCookieJar[Cookie(version=0, name='brain4ce_n', value='vdk5cbnqaj6coqc2jq3gckmk70', port=None, port_specified=False, domain='.edureka.co', domain_specified=True, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=1560820980, discard=False, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False), Cookie(version=0, name='CakeCookie[Visited]', value='%7B%22date%22%3A%222019-05-07+14%3A53%3A00%22%2C%22count%22%3A1%7D', port=None, port_specified=False, domain='www.edureka.co', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=1588843380, discard=False, comment=None, comment_url=None, rest={}, rfc2109=False), Cookie(version=0, name='CakeCookie[currencyprefrence]', value='INR', port=None, port_specified=False, domain='www.edureka.co', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=1559294580, discard=False, comment=None, comment_url=N

In [13]:
req.headers

{'Content-Type': 'text/html; charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Date': 'Tue, 07 May 2019 09:23:00 GMT', 'Server': 'Apache/2.4.27 (Ubuntu)', 'X-Powered-By': 'PHP/5.5.9-1ubuntu4.22', 'Set-Cookie': 'f_l_i_s_p=http%3A%2F%2Fwww.edureka.co%2F; path=/, brain4ce_n=vdk5cbnqaj6coqc2jq3gckmk70; expires=Tue, 18-Jun-2019 01:23:00 GMT; Max-Age=3600000; path=/; domain=edureka.co; HttpOnly, _utm_wb_term=http%3A%2F%2Fwww.edureka.co%2F; expires=Tue, 07-May-2019 10:23:00 GMT; Max-Age=3600; path=/, wb_current=https%3A%2F%2Fwww.edureka.co%2F; expires=Tue, 07-May-2019 10:23:00 GMT; Max-Age=3600; path=/, wb_landing=https%3A%2F%2Fwww.edureka.co%2F; expires=Tue, 07-May-2019 10:23:00 GMT; Max-Age=3600; path=/, CakeCookie[autoLogin]=deleted; expires=Thu, 01-Jan-1970 00:00:01 GMT; Max-Age=0; path=/, CakeCookie[landingpage]=getcoursedata; expires=Fri, 01-May-2043 09:23:00 GMT; Max-Age=756864000; path=/, CakeCookie[preference_country]=12; expires=Fri, 31-May-2019 09:23:00 G

In [14]:
req.elapsed # returns datetime.timedelta(0, 1, 666890)

datetime.timedelta(0, 1, 357481)

In [17]:
import requests
 
query = {'q': 'Forest', 'order': 'popular', 'min_width': '800', 'min_height': '600'}
req = requests.get('https://pixabay.com/en/photos/', params=query)
 
req.url

'https://pixabay.com/images/search/forest/?min_height=600&amp;min_width=800&amp;order=popular'

## MySQL Connection

In [9]:
import pymysql.cursors
import pymysql

# Connect to the database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='root',
                             db='ega_orders',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

try:
    with connection.cursor() as cursor:
        # Read a single record
        sql = "SELECT `order_status` FROM `orders` WHERE `order_id`=%s"
        cursor.execute(sql, (10001,))
        result = cursor.fetchone()
        print(result)
finally:
    connection.close()

{'order_status': 'In shipment'}
